In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [2]:
#get data
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('select * from useducation',con=engine)

In [3]:
#replace missing values
fill = ['ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE', 'TOTAL_EXPENDITURE', 
        'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE',
       'GRADES_PK_G', 'GRADES_KG_G', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G', 'GRADES_9_12_G', 'GRADES_ALL_G']
states = education_df['STATE'].unique()

for state in states:
    education_df.loc[education_df['STATE'] == state, fill] = education_df.loc[education_df['STATE'] == state, fill].interpolate()
#drop na after interpolation
education_df.dropna(inplace = True)

**1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.

In [4]:
#define weighted score variable
education_df['SCORE'] = (education_df['AVG_MATH_4_SCORE'] *
(education_df['GRADES_4_G']/(education_df['GRADES_4_G']+education_df['GRADES_8_G'])) +
education_df['AVG_MATH_8_SCORE'] *
(education_df['GRADES_8_G']/(education_df['GRADES_4_G']+education_df['GRADES_8_G'])) +
education_df['AVG_READING_4_SCORE'] *
(education_df['GRADES_4_G']/(education_df['GRADES_4_G']+education_df['GRADES_8_G'])) +
education_df['AVG_READING_8_SCORE'] *
(education_df['GRADES_8_G']/(education_df['GRADES_4_G']+education_df['GRADES_8_G'])))/2

**2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?**

In [5]:
education_df[['SCORE', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 
             'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,SCORE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
SCORE,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


Instruction Expenditure has the highest correlation with the new overall score variable.

**3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?**

In [6]:
education_df2 = education_df[['INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 
             'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']]
X = StandardScaler().fit_transform(education_df2)
pca = PCA(n_components=1)
education_df['PCA_1'] = pca.fit_transform(X)
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


94% of the total variance in the dataset is explained by the first component from Sklearn PCA.

**4. What is the correlation between the overall score variable and the 1st principal component?**

In [7]:
education_df[['SCORE', 'PCA_1']].corr()

,SCORE,PCA_1
SCORE,1.000000,0.044596
PCA_1,0.044596,1.000000


The correlation between the score variable and the first principle component is 0.044.

**5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?**

The instruction expenditure variable has a stronger correlation with the score variable than the first principle component, so the instruction expenditure variable would be a better feature to use in a model.